## Análise de sentimentos de tweets

Nesta atividade iremos treinar um classificador capaz de determinar se um certo tweet é positivo ou negativo.

Para isso iremos utilizar dados de tweets **EM PORTUGUÊS** já pré-rotulados através da presença de emoticons (carinha triste para negativo e carinha feliz para positivo). O dataset foi isponibilizados pelo monitor e pode ser encontrado [aqui](https://github.com/antonioricardojr/dataset).

Etapas:

1. Importação dos dados.
2. Limpeza e tratamento dos dados.
3. Separação dos dados em treino e teste.
4. Treinamento do classificador.
5. Obtendo tweets de uma hashtag e testando o classificador.

### Importação dos dados

Para importação dos dados que estão em formato csv usaremos a função read_csv do pandas, que já nos devolve os dados no formato de um dataframe.


In [307]:
import pandas as pandas

dados = pandas.read_csv('dados/db.csv',encoding='utf-8', sep='\t')

### Limpeza e tratamento dos dados

Mesmo sem olhar os dados sabemos que existem algumas construções textuais que precisam ser removidas. Links, hashtags e referências a outros usuáris através do '@' são irrelevantes e devem ser removidos. Além disso, devemos remover também *stopwords* e símbolos de pontuação. Temos então a seguinte agenda:

1. Remover links, hashtags e referencias a outros usuários
2. Remover stopwords e símbolos de pontuação


In [308]:
import nltk, re
from string import punctuation as pontuacao
from collections import OrderedDict

def remove_links_hashtags_referencias(texto):
    texto = re.sub(r'http\S+', '', texto)
    texto = re.sub(r'@\S+', '', texto)
    texto = re.sub(r'#\S+', '', texto)
    return texto
    
def remove_stopwords_e_pontuacao(texto):
    texto = remove_links_hashtags_referencias(texto)
    tokens = nltk.word_tokenize(texto.lower())
    stopwords = nltk.corpus.stopwords.words('portuguese')
    return [palavra for palavra in tokens if palavra not in stopwords and palavra not in pontuacao]


### Separação dos dados em treino e teste.

Antes de separar nossos dados é preciso decidir como será feita a extração de *features* dos *tweets*, nesse lab utilizaremos o modelo de *bag-of-words* onde iremos representar cada *tweet* como o conjuto de palavras pelo qual ele é composto.

Com os *bags-of-words* em mãos podemos dividir nossos dados em treino e teste. Iremos utilizar um divisão de 60% dos dados para treino e 40% para teste do nosso modelo.

In [309]:
def obtem_features(text): return dict([(word, True) for word in remove_stopwords_e_pontuacao(text)])

negativos = [(obtem_features(row.text), 'negativo') for _, row in dados[dados.sentiment == 0].iterrows()]
positivos = [(obtem_features(row.text), 'positivo') for _, row in dados[dados.sentiment == 1].iterrows()]


threshold  = .6
num_pos = int(threshold * len(positivos))
num_neg = int(threshold * len(negativos))

dados_treino = positivos[:num_pos] + negativos[:num_neg]
dados_teste = positivos[num_pos:] + negativos[:num_neg]

### Treinamento do classificador

Agora que já preparamos nossos dados podemos treinar nosso classificador utilizando a técnica *Naive Bayes*.

In [310]:
from nltk.classify import NaiveBayesClassifier 
from nltk.classify.util import accuracy as nltk_acc

classifier = NaiveBayesClassifier.train(dados_treino)
print(u'\nAccuracy nos dados de teste: {:.2f}'.format(nltk_acc(classifier, dados_teste)) )

def classifica(texto):
    probs = classifier.prob_classify(obtem_features(texto))
    predicao = probs.max()
    print("{:.02f}% {}".format(probs.prob(predicao)*100, predicao.capitalize()) +  ' - ' + texto)


Accuracy nos dados de teste: 0.86


#### Features mais significativas

Vejamos agora as 20 features(palavras) mais importantes para o classificador determinar o sentimento de um tweet.


In [348]:
classifier.show_most_informative_features(20)

Most Informative Features
                     foo = True           positi : negati =     70.6 : 1.0
                      af = True           negati : positi =     39.3 : 1.0
                    veja = True           positi : negati =     29.2 : 1.0
                   breve = True           positi : negati =     26.7 : 1.0
                   bravo = True           positi : negati =     25.4 : 1.0
                   clube = True           positi : negati =     21.7 : 1.0
                     vei = True           negati : positi =     21.6 : 1.0
                   segui = True           positi : negati =     21.0 : 1.0
                  adorei = True           positi : negati =     21.0 : 1.0
                     bar = True           negati : positi =     19.9 : 1.0
                     sdd = True           negati : positi =     19.7 : 1.0
                 saudade = True           negati : positi =     18.0 : 1.0
                  triste = True           negati : positi =     17.5 : 1.0

O fato dos dados de treino terem sido obtidos e classificados e positivos através da presença de emoticons afetou bastante o treino do classificador. Muitas vezes uma carinha feliz no âmbito da internet pode denotar sarcasmo ou ironia com relação a algum fato *negativo*. A palavra 'bravo' é um exemplo disso, caso ela apareça um tweet este tem 25 vezes mais chances de ser *positivo*, algo no mínimo estranho para uma palavra que tem uma semântica inerentemente negativa.

Vejamos um exemplo:

In [363]:
classifica('Hoje estou muito bravo, o dia foi horrível.')
classifica('O clube de futebol São Paulo sofreu sua pior derrota da história')
    

92.70% Positivo - Hoje estou muito bravo, o dia foi horrível.
71.20% Positivo - O clube de futebol São Paulo sofreu sua pior derrota da história


Contudo, nosso classificador parece se comportar como esperado em cenários específicos como:


In [365]:
classifica('Hoje eu vou para o bar beber ate passar a saudade dela.')
classifica('Adorei o novo filme dos vingadores')

99.74% Negativo - Hoje eu vou para o bar beber ate passar a saudade dela.
96.73% Positivo - Adorei o novo filme dos vingadores


### Testando o classificador com novos tweets

Para testar o classificador eu obtive alguns tweets da hashtag [#LulaPreso](https://twitter.com/hashtag/lulapreso?lang=en). Vamos ver o que o nosso classificador tem a dizer sobre eles.

In [364]:
tweets = []

tweets.append('#LulaLivre ? Só que não. Lula bom é lula morto, trocando em miúdos é uma apologia à corrupção e ao crime. A hashtag da vez deve ser #LulaPreso porque lugar de bandido é na cadeia')
tweets.append('O MEU MOVIMENTO É: #LULAPRESO SE MANTER A JARARACA DENTRO DA GARRAFA DE CACHAÇA ELA NÃO MORDERÁ NINGUÉM E O BRASIL SERÁ FELIZ. ')
tweets.append('COM O MOLUSCO CACHACEIRO NA CADEIA A BOSTA DESSE PAÍS VAI PRA FRENTE #LULAPRESO')

for tweet in tweets:
    classifica(tweet)

81.57% Negativo - #LulaLivre ? Só que não. Lula bom é lula morto, trocando em miúdos é uma apologia à corrupção e ao crime. A hashtag da vez deve ser #LulaPreso porque lugar de bandido é na cadeia
67.29% Positivo - O MEU MOVIMENTO É: #LULAPRESO SE MANTER A JARARACA DENTRO DA GARRAFA DE CACHAÇA ELA NÃO MORDERÁ NINGUÉM E O BRASIL SERÁ FELIZ. 
92.98% Negativo - COM O MOLUSCO CACHACEIRO NA CADEIA A BOSTA DESSE PAÍS VAI PRA FRENTE #LULAPRESO


### Considerações finais

A abordagem utilizada para pré-classificar os dados de treino poderia ser mais robusta, talvez utilizando alguma alguma técnica de processamento de linguagem natural.

Poderiamos usar um volume maior de dados para treinar pois assim palavras que tiveram resultados inesperados como 'bravo' possivelmente iriam tender ao seu real significado semântico.

Ainda é possível aprimorar o pré-processamento dos textos dos tweets usando técnicas como *stemming* (reduzir verbos ao infinitivo é um exemplo). Nesse lab eu tentei utilizar o SnowballStemmer do nltk mas não obtive resultados satisfatórios.

